In [1]:
%pylab inline
import os
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.parameter import ModelParameters
from Chempy.cem_function import single_timestep_chempy
a=ModelParameters()
from Chempy.sfr import SFR

age_of_star=10

new_timesteps = int((a.time_steps-1)*a.end/(a.end-age_of_star)+1)
new_end = (a.end-age_of_star)*(new_timesteps-1)/(a.time_steps-1)

## SHOULD WE NORMALIZE BY NEW OR OLD END TIME?
basic_sfr = SFR(a.start,new_end,new_timesteps)

old_sfr = SFR(a.start,a.end,a.time_steps)

if a.basic_sfr_name == 'gamma_function':
    getattr(basic_sfr, a.basic_sfr_name)(S0 = a.S_0 * a.mass_factor,a_parameter = a.a_parameter, loc = a.sfr_beginning, scale = a.sfr_scale)
    getattr(old_sfr, a.basic_sfr_name)(S0 = a.S_0 * a.mass_factor,a_parameter = a.a_parameter, loc = a.sfr_beginning, scale = a.sfr_scale)


# compute a small correction from changing the simulation end time
from scipy.stats import gamma
basic_gamma = np.sum(gamma.pdf(basic_sfr.t,a.a_parameter,a.sfr_beginning,a.sfr_scale))*basic_sfr.dt
old_gamma = np.sum(gamma.pdf(old_sfr.t,a.a_parameter,a.sfr_beginning,a.sfr_scale))*old_sfr.dt
correction = basic_gamma/old_gamma
a.total_mass*=correction

## NB: SFR is a discrete quantity showing the amount of mass formed in each time bin
basic_sfr.sfr = a.total_mass * np.divide(basic_sfr.sfr,sum(basic_sfr.sfr))
mass_normalisation = a.total_mass
## mean_sfr is an actual RATE unlikfe .sfr 
mean_sfr = sum(basic_sfr.sfr) / new_end

# at which time in the simulation is the star born
star_time = a.end-age_of_star#basic_sfr.t[-1] - age_of_star
cut = [np.where(np.abs(basic_sfr.t - star_time) == np.min(np.abs(basic_sfr.t - star_time)))]
if len(cut[0][0]) != 1:
    cut = cut[0][0][0]

Populating the interactive namespace from numpy and matplotlib


In [2]:
# updating the end time and the model steps and rescale the total mass
time_model = float(basic_sfr.t[tuple(cut)])
a.end = time_model
a.time_steps = int(cut[0][0]) + 1


## this is new amount of mass formed in this time period (dimensionless)
a.total_mass = sum(basic_sfr.sfr[0:a.time_steps])


In [3]:
# check whether the sfr is enough at end to produce reasonable number of stars (which is necessary in order to have a probability to observe a star at all)
sfr_at_end = float(basic_sfr.sfr[tuple(cut)] / basic_sfr.dt) # convert to an actual rate
fraction_of_mean_sfr = sfr_at_end / mean_sfr

a.shortened_sfr_rescaling = a.total_mass / mass_normalisation	

if fraction_of_mean_sfr<0.05:
    print("bad")
else:
    pass
#assert fraction_of_mean_sfr > 0.05, ('The total SFR of the last age bin is below 5% of the mean SFR', 'stellar identifier = ', a.stellar_identifier, 'star time = ', star_time, 'model time = ', time_model ) 

## Check where a.end is used later?